In [ ]:
%load_ext autoreload
%autoreload 2
%store -r the_editor

if 'the_editor' not in locals():
    import pickle
    print("Loading default data...")
    the_editor = pickle.load(open("data/the_editor.p",'rb'))
    
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# A. Basic editor information (from Wikipedia)"))
display(md(f"***Editor: {the_editor['name']}***"))

In [ ]:
the_editor.to_frame('value')

In [ ]:
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# B. Modified pages of an editor (from Xtools)"))
display(md(f"***Editor: {the_editor['name']}***"))

In [ ]:
from external.xtools import XtoolsAPI, XtoolsDV
xtools_api = XtoolsAPI(project = 'en.wikipedia.org')
xtools_dv = XtoolsDV(xtools_api)
editor_info = xtools_dv.get_modified_pages_counts_per_editor(the_editor['name'])
editor_info.to_frame('value')

In [ ]:
from IPython.display import display, Markdown as md
display(md("---"))
display(md(f"# C. Select one page created by an editor (XTools)"))
display(md(f"***Editor: {the_editor['name']}***"))

In [ ]:
from external.wikipedia import WikipediaDV, WikipediaAPI
from IPython.display import HTML
from utils.notebooks import get_notebook_by_number
from external.xtools import XtoolsAPI, XtoolsDV

xtools_api = XtoolsAPI(project = 'en.wikipedia.org')
xtools_dv = XtoolsDV(xtools_api)
created_pages = xtools_dv.get_created_pages_per_editor(the_editor['name'])

from visualization.conflicts_listener import ConflictsListener
def select_page(page):
    global the_page
    page_inputname=page
    
    wikipedia_dv = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))
    try:
        the_page = wikipedia_dv.get_page(int(page_inputname))
    except:
        the_page = wikipedia_dv.get_page(page_inputname)

    with out:
        %store the_page
        clear_output()
        display(md("### Current Selection:"))
        
        if 'invalid' in the_page:
            display(f"The page {page_inputname} was not found, try a different page")
        else:
            # display the data that will be passed to the next notebook
            display(the_page.to_frame('values'))
            display(HTML(f'<a href="{get_notebook_by_number(1)}" target="_blank">Go to next workbook</a>'))
   

def on_selection_change(change):
    try:
        select_page(qg_obj.get_selected_df().iloc[0].page_title)
    except:
        print('Problem parsing the name. Execute the cell again and try a different page.')


import qgrid
qg_obj = qgrid.show_grid(created_pages[['page_title', 'page_len', 'rev_id', 'rev_len', 'rev_timestamp']])
qg_obj.observe(on_selection_change, names=['_selected_rows'])

from ipywidgets import Output
from IPython.display import display, clear_output, Markdown as md
display(md("### Select one page row for the next notebook:"))
display(qg_obj)
out = Output()
display(out)

